# Assignment 6

First we have to create some SQL tables.

In [1]:
CREATE DATABASE MouseHypothalamus;

Commands completed successfully.

Total execution time: 00:00:02.609

In [7]:
USE MouseHypothalamus;
DROP TABLE IF EXISTS #TempMolecules;
CREATE TABLE #TempMolecules(
    Gene_name NVARCHAR(10) NOT NULL,
    Cell_name NVARCHAR(40),
    Animal_ID int,
    Bregma float,
    Animal_sex nvarchar(6) NOT NULL,
    Behavior nvarchar(10),
    Centroid_X float NOT NULL,
    Centroid_Y float NOT NULL,
    Centroid_Z float NOT NULL,
    Total_brightness float,
    Area int,
    Error_bit tinyint,
    Error_direction tinyint
);

BULK INSERT #TempMolecules FROM '/var/data/merfish_barcodes.csv'
WITH ( 
    FIRSTROW = 2, -- skip the column headers
    ROWS_PER_BATCH = 467052741, -- however many total rows the data has
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '0x0a',
    KEEPNULLS
);


The statement has been terminated.

Query was canceled by user

Total execution time: 00:03:49.983

TODO: Add a column for the identity (auto-increment).

In [ ]:
Use MouseHypothalamus;
SELECT IDENTITY(int,1,1) as id,* INTO Molecules FROM #TempMolecules;